In [9]:
import scipy.io
import numpy as np
from sklearn.metrics import accuracy_score
import functional as F
import utils as ut
import time


In [10]:
start_time = time.time()
data = scipy.io.loadmat('../data/TsukubaHandSize24x24.mat')
data = data['data']

train_data, test_data = ut.split_train_test(data)
train_data = ut.reshape_data(train_data, 80)
test_data = ut.reshape_data(test_data, 20)

train_data = ut.split_and_reshape(train_data)
test_data = ut.split_and_reshape(test_data)

# plot train images
# first_images = [cls_arr[:, :, 0] for cls_arr in X_train]
# plot_first_images(first_images)

# # plot test images
# first_images = [cls_arr[:, :, 0] for cls_arr in X_test]
# plot_first_images(first_images)

train_data = ut.flatten_and_transpose(train_data)
test_data = ut.flatten_and_transpose(test_data)

print('Train data shape ',np.array(train_data).shape)
print('Test data shape ',np.array(test_data).shape)

# Fianl preparation of the dataset
NUM_OF_CLASSES = 30
NUM_OF_SETS = 5

X_train = [_X.T for _X in train_data]  # transpose into (N=896, M=30)

test_sets = []

for test_class in test_data:
    test_sets.extend(np.array_split(test_class, NUM_OF_SETS))

X_test = [_X.T for _X in test_sets]
y_test = []

for i, _ in enumerate(range(NUM_OF_CLASSES)):
    y_test.extend(NUM_OF_SETS * [i])  # add corresponding labels into y_test

Train data shape  (30, 33600, 576)
Test data shape  (30, 8400, 576)


In [11]:
# Creating the reference subspace
n_dims = 10
ref_subspaces = [F.pca(_X, n_dims=n_dims) for _X in X_train]
ref_subspaces = np.array(ref_subspaces)
print(f"Reference subspaces tensor shape: {ref_subspaces.shape}")


# Create the input subpsace
input_subspaces = [F.pca(set, n_dims=n_dims) for set in X_test]
print(f"Input subspaces shape: {input_subspaces[0].shape}")
print(f"Input subspaces len: {len(input_subspaces)}")

# Calculate the similarities of the input and the reference subspaces
similarities = np.array(
    [
        F.canonical_similarity(ref_subspaces, input_subspace)
        for input_subspace in input_subspaces
    ]
)
predictions = np.argmax(similarities, axis=1)


acc = accuracy_score(y_test, predictions)
end_time = time.time()

elapsed_time = end_time - start_time
minutes, seconds = divmod(elapsed_time, 60)

print("Time took: {:0>2}:{:05.2f}".format(int(minutes), seconds))
print(f"MSM accuracy: {acc * 100:.2f}")

Reference subspaces tensor shape: (30, 576, 10)
Input subspaces shape: (576, 10)
Input subspaces len: 150
Time took: 00:26.29
MSM accuracy: 89.33
